In [ ]:
#!pip install dippykit

In [34]:
import numpy as np
import cv2
import zmq
import time
#from statistics import stdev
from utils import *
from dippykit.coding import huffman_encode
from dippykit.coding import huffman_decode
import pickle
from matplotlib import pyplot as plt
import zlib
import base64

img = cv2.imread('beer.png')

In [35]:
R = Retina(gpu=False)
R.loadLoc('/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/retina_data/ret50k_loc.pkl')
R.loadCoeff('/Users/nicholassperry/Desktop/Year 4.nosync/Pipeline/retina_data/ret50k_coeff.pkl')
fixation = (1080/2,1920/2)
shape = (1080, 1920)
R.prepare(shape, fixation)

V = R.sample(img, fixation).astype('uint8')

In [44]:
def send_image(n=5):
    video_path = '/Users/nicholassperry/Desktop/Dataset.nosync/Dataset/Train/Gargoyle.MOV'
    vidcap = cv2.VideoCapture(video_path)
    success, img = vidcap.read()

    count = 1
    pickling_times = []
    sending_times = []
    total_start = time.time()
    while success:
        if count%n == 0:
            break
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        t1 = time.time()
        
        data = pickle.dumps(img, protocol=-1)
        data = zlib.compress(data, 6)
        
        t2 = time.time()
        
        tracker = socket.send(data, copy=False, track=True)
        while tracker.done == False:
            pass
            
        t3 = time.time()
        
        pickling_times.append(t2-t1)
        sending_times.append(t3-t2)
        
        success, img = vidcap.read()
        count += 1
    total_end = time.time()
    socket.send(b'Done')
    socket.send_string(str(total_end-total_start))
    
    print('Successfully converted video to %d frames'%count)
    
    print('\nProcessing time: %f'%sum(pickling_times))    
    print('Time taken to send: %f'%sum(sending_times))
    print('Overall total time: %f'%(total_end-total_start))

In [56]:
def send_full_vector(n=5):
    video_path = '/Users/nicholassperry/Desktop/Dataset.nosync/Dataset/Train/Gargoyle.MOV'
    vidcap = cv2.VideoCapture(video_path)
    success, img = vidcap.read()

    count = 1
    processing_times = []
    sending_times = []
    total_start = time.time()
    
    while success:
        print(count)
        if count%n == 0:
            break
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        t1 = time.time()
        
        V = R.sample(img, fixation).astype('uint8')
        V = pickle.dumps(V, protocol=-1)
        #V = V.tostring()
        
        t2 = time.time()
        
        tracker = socket.send(V, copy=False, track=True)
        while tracker.done == False:
            pass
            
        t3 = time.time()

        processing_times.append(t2-t1)
        sending_times.append(t3-t2)
        
        success, img = vidcap.read()
        count += 1
    total_end = time.time()
    print('Successfully converted video to %d frames'%count)
    
    print('\nProcessing time: %f'%sum(processing_times))    
    print('Time taken to send: %f'%sum(sending_times))
    print('Overall total time: %f'%(total_end-total_start))

def send_encoded_vector(n):
    video_path = '/Users/nicholassperry/Desktop/Dataset.nosync/Dataset/Train/Gargoyle.MOV'
    vidcap = cv2.VideoCapture(video_path)
    success, img = vidcap.read()

    count = 1
    huffman_times = []
    sampling_times = []
    sending_times = []
    total_start = time.time()
    while success:
        print(count, end='\r')
        if count%n == 0:
            break
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        t1 = time.time()
        
        V = R.sample(img, fixation).astype('uint8')
        
        t2 = time.time()
        
        encoded_V, bits, code_dict, _ = huffman_encode(V.flatten())
        data = [encoded_V, code_dict, bits]
        data = pickle.dumps(data, protocol=-1)
        #data = zlib.compress(data, 6)
        
        t3 = time.time()
        
        tracker = socket.send(data, copy=False, track=True)
        while tracker.done == False:
            pass

        t4 = time.time()
        
        sampling_times.append(t2-t1)
        huffman_times.append(t3-t2)
        sending_times.append(t4-t3)
        success, img = vidcap.read()
        count += 1
    total_end = time.time()
    
    socket.send(b'Done')
    socket.send_string(str(total_end-total_start))
    
    print('Successfully converted video to %d frames'%count)
    
    print('\nSampling time: %f'%sum(sampling_times))
    print('Huffman encoding time: %f'%sum(huffman_times))
    print('Time taken to send: %f'%sum(sending_times))
    print('Overall total time: %f'%(total_end-total_start))

In [46]:
context = zmq.Context()

#  Socket to talk to server
print("Connecting to Server…")
socket = context.socket(zmq.PUSH)
socket.connect("tcp://localhost:5555")
time.sleep(2)

socket.send(b'Normal')
send_image(n=5)

socket.close()

Connecting to Server…
Successfully converted video to 5 frames

Processing time: 0.752174
Time taken to send: 0.033682
Overall total time: 0.817037


In [57]:
context = zmq.Context()

#  Socket to talk to server
print("Connecting to Server…")
socket = context.socket(zmq.PUSH)
socket.connect("tcp://localhost:5555")

socket.send(b'Cort')
send_encoded_vector(n=3)

socket.close()

Connecting to Server…
Successfully converted video to 3 frames

Sampling time: 6.692271
Huffman encoding time: 0.191236
Time taken to send: 0.013533
Overall total time: 6.911135


In [ ]:
context = zmq.Context()

#  Socket to talk to server
print("Connecting to Server…")
socket = context.socket(zmq.PUSH)
socket.connect("tcp://localhost:5555")

socket.send(b'Cort')
send_full_vector(n=2)

socket.close()

In [ ]:
context = zmq.Context()

print("Connecting to Server")
client_socket = context.socket(zmq.PUSH)
client_socket.connect("tcp://localhost:5555")

for i in range(10):
    client_socket.send(b'Hello')